## ESG Research 
### Yfinance, Pandas, NumPy


# Installation
* pip install pandas
* pip install numpy
* pip install yfinance | pip3 install yfinance
* pip install git+https://github.com/rodrigobercini/yfinance.git


In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
from pandas import Series, DataFrame
from pandas.tseries import offsets
import csv
import datetime
import csv
from csv import writer

import os

dt = datetime.datetime.today()

YEAR = dt.year
MONTH = dt.month
CURRENT_CSV_FILE = bool

ESG_COLUMNS = {'palmOil', 'controversialWeapons', 'gambling', 'socialScore', 'nuclear',
       'furLeather', 'alcoholic', 'gmo', 'catholic', 'socialPercentile',
       'peerCount', 'governanceScore', 'environmentPercentile',
       'animalTesting', 'tobacco', 'totalEsg', 'highestControversy',
       'esgPerformance', 'coal', 'pesticides', 'adult', 'percentile',
       'peerGroup', 'smallArms', 'environmentScore', 'governancePercentile',
       'militaryContract', 
       #'Stock'
}

ALL_COLUMNS = {'palmOil', 'peerCount', 'environmentScore', 'militaryContract',
       'esgPerformance', 'coal', 'peerGroup', 'furLeather', 'gambling',
       'animalTesting', 'catholic', 'nuclear', 'totalEsg', 'adult',
       'environmentPercentile', 'highestControversy', 'socialScore',
       'percentile', 'alcoholic', 'socialPercentile', 'pesticides',
       'governancePercentile', 'controversialWeapons', 'gmo', 'smallArms',
       'tobacco', 'governanceScore', 'Stock Ticker', 'Sector', 'Name'
}

USER_PATH = '/Users/MichaelCalmette/Documents/Finance Research/ESG Data Files/'
DATE_PATH = "esg-{}-{}.csv".format(dt.month, dt.year)
FILE_PATH = os.path.join(USER_PATH,DATE_PATH)

In [2]:
f = open(FILE_PATH, 'w')
f.close

<function TextIOWrapper.close()>

Need to compare data in esg-csv to all tickers

In [3]:
df = pd.read_csv('tickers.csv')
all_data = pd.DataFrame(columns=ESG_COLUMNS)

In [4]:
for x in range(2):
    tic = df.sample(n=1)
    ticker_sym = tic.loc[tic.index[0],'Symbol'] # get ticker name ex: 'AAPL'
    print(ticker_sym)
    ticker = yf.Ticker(ticker_sym)
    y = ticker.sustainability 
    
    #df1_transposed = y.T
    df1_transposed = y
    df1_transposed = df1_transposed.transpose()
    #print(df1_transposed.columns)
    df1_transposed['Stock Ticker'] = ticker_sym
    df1_transposed['Sector'] = tic.loc[tic.index[0],'Sector']
    df1_transposed['Name'] = tic.loc[tic.index[0],'Name']

    #all_data = df1_transposed
    all_data = all_data.append(df1_transposed, ignore_index = True)

MDT
Index(['palmOil', 'controversialWeapons', 'gambling', 'socialScore', 'nuclear',
       'furLeather', 'alcoholic', 'gmo', 'catholic', 'socialPercentile',
       'peerCount', 'governanceScore', 'environmentPercentile',
       'animalTesting', 'tobacco', 'totalEsg', 'highestControversy',
       'esgPerformance', 'coal', 'pesticides', 'adult', 'percentile',
       'peerGroup', 'smallArms', 'environmentScore', 'governancePercentile',
       'militaryContract'],
      dtype='object', name='2022-2')
KHC
Index(['palmOil', 'controversialWeapons', 'gambling', 'socialScore', 'nuclear',
       'furLeather', 'alcoholic', 'gmo', 'catholic', 'socialPercentile',
       'peerCount', 'governanceScore', 'environmentPercentile',
       'animalTesting', 'tobacco', 'totalEsg', 'highestControversy',
       'esgPerformance', 'coal', 'pesticides', 'adult', 'percentile',
       'peerGroup', 'smallArms', 'environmentScore', 'governancePercentile',
       'militaryContract'],
      dtype='object', name='2022-

In [5]:
all_data

,environmentScore,governanceScore,governancePercentile,pesticides,peerCount,adult,esgPerformance,socialPercentile,catholic,totalEsg,...,coal,socialScore,alcoholic,nuclear,smallArms,animalTesting,gambling,Stock Ticker,Sector,Name
0,2.34,7.66,None,False,58,False,AVG_PERF,None,False,23.39,...,False,13.39,False,False,False,True,False,MDT,Health Care,Medtronic
1,13.09,6.75,None,False,94,False,OUT_PERF,None,False,34.47,...,False,14.63,False,False,False,True,False,KHC,Consumer Staples,Kraft Heinz


In [7]:
if os.stat(FILE_PATH).st_size == 0:
    CURRENT_CSV_FILE = 0
    print('ESG File is empty')
    f = open(FILE_PATH, 'w')
    f.close

    with open(FILE_PATH, 'a', newline='') as f_object: 
        writer_object = writer(f_object) # Pass the CSV  file object to the writer() function
        writer_object.writerow(all_data) # Pass the data in the list as an argument into the writerow() function
        f_object.close()
    print('Column names have been entered.')

else:
    CURRENT_CSV_FILE = 1
    print('ESG File has column headers.')
    all_data.to_csv(FILE_PATH, mode='a', index=False, header=False)
    print('ESG Data has been entered.')

ESG File has column headers.
ESG Data has been entered.


Download S&P 500 tickers from csv file: *tickers.csv*

Get a random ticker from the dataset using *sample()*

In [ ]:

def data_passer():
    for x in range(2):
        tic = df.sample(n=1) #get random ticker
        ticker_sym = tic.loc[tic.index[0],'Symbol'] #get ticker symbol
        ticker = yf.Ticker(ticker_sym) #define in yfinance
        y = ticker.sustainability  #get 
        y.head()
        df1_transposed = y.T

        #y = y.rename(columns={'Value': ticker_sym})
        #df2 = df1_transposed
        #df2
        print(ticker_sym)

    return y


Now, we want to write the data into the csv file to build up the database

In [ ]:
import csv

# open the file in the write mode
f = open('/Users/MichaelCalmette/Documents/Finance Research/esg-data.csv', 'w')

# create the csv writer
writer = csv.writer(f)

# write a row to the csv file
writer.writerow(df1_transposed)

# close the file
f.close()

In [ ]:
from csv import writer
  
# The data assigned to the list 

with open('data.csv', 'a', newline='') as f_object:  
    writer_object = writer(f_object) # Pass the CSV  file object to the writer() function
    writer_object.writerow(list_data) # Pass the data in the list as an argument into the writerow() function
    f_object.close()


#/Users/MichaelCalmette/Documents/Finance Research/ESG-yfinance.ipynb
